In [1]:
import pred_location as pl

In [1]:
import locate_object as lo
import os
import json
import subprocess

from tensorbox_model import TensorBox

import tensorflow as tf

import train

In [2]:
_json_format = '[\n{\n"rects": [\n  {\n   "y1": 100.0,\n   "x2":  150.0,\n   "x1":  100.0,\n   "y2": 150.0\n  }\n ],\n "image_path": "%s"\n}\n]\n'
_json_format2= '[\n{\n"rects": [\n  {\n   "y1": 100.0,\n   "x2":  150.0,\n   "x1":  100.0,\n   "y2": 150.0\n  }\n ],\n "image_path": "%s"\n},'+\
                '\n{\n"rects": [\n  {\n   "y1": 200.0,\n   "x2":  250.0,\n   "x1":  200.0,\n   "y2": 250.0\n  }\n ],\n "image_path": "%s"\n}\n]\n'

def _generate_json( inp_file_path ):
    
    with open('sample.json','w') as f:
#        f.write( _json_format % inp_file_path )
        f.write( _json_format2 % (inp_file_path,inp_file_path) )
        
def get_rect( inp_file_name ):

    _generate_json( inp_file_name )
    
    _weights    = 'nn_model/save.ckpt-5000'
    _test_boxes = 'sample.json'
    _min_conf   = 0.2
    _tau        = 0.25
    _show_supp  = True
    _iou_thresh = 0.5
    
    os.environ['CUDA_VISIBLE_DEVICES'] = str(0)
    hypes_file = '%s/hypes.json' % os.path.dirname( _weights )
    with open(hypes_file, 'r') as f:
        H = json.load(f)
    expname = ''
    pred_boxes = '%s.%s%s' % ( _weights, expname, os.path.basename( _test_boxes))
    true_boxes = '%s.gt_%s%s' % ( _weights, expname, os.path.basename( _test_boxes))

    
    tensorbox = TensorBox(H)
    rect_list = tensorbox.pred( _weights,
                                                   _test_boxes,
                                                   _min_conf,
                                                   _tau,
                                                   _show_supp,
                                                  expname)

    print rect_list


In [3]:
obj_dir  = '/home/sean/Insight/legos/tensorbox/'
img_name = 'sample_lego_brick_2.jpg'

In [4]:
tf.reset_default_graph()
get_rect( obj_dir+img_name )

INFO:tensorflow:Restoring parameters from nn_model/save.ckpt-5000


/home/sean/anaconda/lib/python2.7/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/home/sean/anaconda/lib/python2.7/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


train_utils 2
train_utils again 1
tb model  1
0
/home/sean/Insight/legos/tensorbox/sample_lego_brick_2.jpg
train_utils 2
train_utils again 1
tb model  1
/home/sean/Insight/legos/tensorbox/sample_lego_brick_2.jpg
[{'x_min': 0.21884765625, 'y_max': 0.846875, 'y_min': 0.096875, 'x_max': 0.78662109375}, {'x_min': 0.21884765625, 'y_max': 0.846875, 'y_min': 0.096875, 'x_max': 0.78662109375}]


In [1]:
# New tensorbox libraries
import tensorflow as tf
import os, json, subprocess, random
from optparse import OptionParser
from os import path

from scipy.misc import imread, imresize, imsave
import numpy as np
from PIL import Image, ImageDraw

from train import build_forward
from utils.annolist import AnnotationLib as al
from utils.train_utils import add_rectangles, rescale_boxes
from utils.data_utils import Rotate90

In [ ]:

_weights_full_path = '/home/sean/Insight/other_tensorbox/TensorBox/lego_train/overfeat_rezoom_2018_06_14_09.20/save.ckpt-1500'
_hypes_full_path   = '/home/sean/Insight/tensorbox/hypes/hypes.json'

def new_main( inp_image_path ):
    
    image_filename = inp_image_path
    hypes_path     = _hypes_full_path
    weights_path   = _weights_full_path

    parser = OptionParser(usage='usage: %prog [options] <image> <hypes>')
    parser.add_option('--gpu', action='store', type='int', default=0)
    parser.add_option('--tau', action='store', type='float', default=0.25)
    parser.add_option('--min_conf', action='store', type='float', default=0.2)

    (options, args) = parser.parse_args()
    if len(args) < 2:
        print ('Provide path configuration json file')
        return

    config = json.load(open(hypes_path, 'r'))
    
    
    
    init_params = initialize(weights_path, hypes_path, options)
    init_params['pred_options'] = {'verbose': True}
#    pred_anno = hot_predict(image_filename, init_params)
#    save_results(image_filename, pred_anno, 'predictions_sliced')